### Modificaciones necesarias en base a conclusiones del analisis de calidad
***Google***
- Eliminar filas duplicadas y filas de negocios que ya cerraron
- Eliminar columnas sobrantes y con muchos nulos (price,hours,relative_results y url)

In [2]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

In [3]:
# Ruta del archivo .pkl
folder_path = r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\Proyecto Final\datasets\Google Maps\metadata-sitios'

# Obtiene los todos los archivos de la carpeta
files = os.listdir(folder_path)
dataframes = []

# Leer cada archivo JSON línea por línea
for file_name in files:
    if file_name.endswith('.json'):  # Asegurarse de que solo se lean los archivos JSON
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_json(file_path, lines=True)  # Leer el archivo JSON línea por línea
            dataframes.append(df)
        except ValueError as e:
            print(f"Error leyendo {file_name}: {e}")

# Concatenar todos los DataFrames en uno solo
df_business_google_base = pd.concat(dataframes, ignore_index=True)

In [4]:
df_business_google = df_business_google_base

### Desanidamos Misc

In [5]:
# 1. Desanidar la columna 'MISC' usando pd.json_normalize
misc_expanded = pd.json_normalize(df_business_google['MISC'])

# 2. Combinar las nuevas columnas con el DataFrame original
df_business_google = pd.concat([df_business_google, misc_expanded], axis=1)

# 3. Eliminar la columna original 'MISC' si ya no es necesaria
df_business_google = df_business_google.drop(columns=['MISC'])

# 4. Convertir listas en las nuevas columnas a cadenas separadas por comas
# Aseguramos que cada elemento dentro de las listas se convierta a string
df_business_google = df_business_google.applymap(
    lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x
)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_15712\459635579.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_business_google = df_business_google.applymap(


KeyboardInterrupt: 

### Hacemos transformaciones

In [6]:
# 2. Eliminar las columnas price','hours','relative_results','url'
df_business_google = df_business_google.drop(columns=['price','hours','url'])

# 1. Eliminar filas duplicadas cuando toda la fila es idéntica
df_business_google = df_business_google.drop_duplicates()

# 3. Asegurarse de que la columna 'state' esté convertida a texto (string)
df_business_google['state'] = df_business_google['state'].astype(str)

# 4. Eliminar filas donde 'state' es igual a 'Permanently closed'
df_business_google = df_business_google[df_business_google['state'] != 'Permanently closed']

# 5. Verificar el resultado final
print(df_business_google.head())

# También podrías querer verificar el número de filas después de los cambios
print(f"El DataFrame tiene {df_business_google.shape[0]} filas después de la limpieza.")


               name                                            address  \
0   Porter Pharmacy  Porter Pharmacy, 129 N Second St, Cochran, GA ...   
1      City Textile  City Textile, 3001 E Pico Blvd, Los Angeles, C...   
2      San Soo Dang  San Soo Dang, 761 S Vermont Ave, Los Angeles, ...   
3      Nova Fabrics  Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...   
4  Nobel Textile Co  Nobel Textile Co, 719 E 9th St, Los Angeles, C...   

                                 gmap_id description   latitude   longitude  \
0  0x88f16e41928ff687:0x883dad4fd048e8f8        None  32.388300  -83.357100   
1  0x80c2c98c0e3c16fd:0x29ec8a728764fdf9        None  34.018891 -118.215290   
2  0x80c2c778e3b73d33:0xbdc58662a4a97d49        None  34.058092 -118.292130   
3   0x80c2c89923b27a41:0x32041559418d447        None  34.023669 -118.232930   
4  0x80c2c632f933b073:0xc31785961fe826a6        None  34.036694 -118.249421   

            category  avg_rating  num_of_reviews              state  ...  \
0   

## Filtramos solo datos de california mediante el uso de las claves gmap_id de reviews-California


Primero obtenemos la columna gmap_id de reviews y luego filtramos los datos del df 

In [69]:
# Cargar la columna 'gmap_id' desde el archivo Parquet
parquet_path = r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\data parcial\reviews_google.parquet'
gmap_id_california = pd.read_parquet(parquet_path, columns=['gmap_id'])

# Filtrar los registros en df_business_google que coincidan en la columna 'gmap_id'
df_business_google = df_business_google[df_business_google['gmap_id'].isin(gmap_id_california['gmap_id'])]


In [ ]:
df_business_google

In [70]:
len(df_business_google)

7923

In [13]:
len(gmap_id_california)

2624574

### Se agrega el acceso el archivo para evitar pasos anteriores en cada ejecución

In [71]:
import pandas as pd
df_business_google = pd.read_parquet(r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\data parcial\business_google.parquet')

In [72]:
len(df_business_google)

67578

In [ ]:
df_business_google['category'].fillna('Valores Faltantes', inplace=True)

In [ ]:
# Definir el patrón de expresión regular para identificar categorías relacionadas con la gastronomía
palabra = r'\b(restaurant|cafe|delivery|diner|bistro|takeout|bar|pub|grill|pizzeria|coffee|bakery|food|eatery|sandwich|snack)\b'

# Filtrar las categorías que coinciden con el patrón de gastronomía, manejando NaN
df_business_google = df_business_google[df_business_google['category'].str.contains(palabra, case=False, regex=True, na=False)]


In [ ]:
df_business_google

## Filtramos para que queden solo locales de comida

In [75]:
#df_business_google['category'] = df_business_google['category'].str.split(',').str[0]

group = {
    'Restaurant': ['Restaurant'],
    'Fast Food': ['Fast food restaurant', 'Pizza restaurant', 'Pizza Takeout'],
    'Takeout/Delivery': ['Takeout Restaurant', 'Delivery Restaurant'],
    'Mexican Cuisine': ['Mexican restaurant', 'Taco restaurant'],
    'American Cuisine': ['American restaurant', 'Burger restaurant'],
    'Sandwich Shop': ['Sandwich shop'],
    'Italian Cuisine': ['Italian restaurant'],
    'Chinese Cuisine': ['Chinese restaurant'],
    'Coffee Shop': ['Coffee shop', 'Cafe'],
    'Seafood Restaurant': ['Seafood restaurant'],
    'Barbecue Restaurant': ['Barbecue restaurant'],
    'Asian Cuisine': ['Asian restaurant', 'Sushi restaurant', 'Japanese restaurant', 'Thai restaurant'],
    'Chicken Restaurant': ['Chicken restaurant', 'Chicken wings restaurant'],
    'Bakery': ['Bakery', 'Bakery shop'],
    'Ice Cream Shop': ['Ice cream shop'],
    'Indian Cuisine': ['Indian restaurant'],
    'Latin American Cuisine': ['Latin American restaurant'],
    'Juice Shop': ['Juice shop'],
    'Specialty Dining': ['Caterer'],
    'Vegetarian restaurant': ['Vegetarian restaurant'],
    'Vietnamese Restaurant': ['Vietnamese Restaurant'],
    'Health Food' : ['Health food store', 'Health Food', 'Green Food']
}

def assign_group(category):
    if pd.isna(category):  # Si la categoría es None o NaN
        return 'Other'
    for key, values in group.items():
        for value in values:
            if value.lower() in category.lower():
                return key
    return 'Other'

df_business_google['group'] = df_business_google['category'].map(assign_group)

In [76]:
df_business_google = df_business_google[df_business_google['group'] != 'Other']

In [ ]:
df_business_google

In [77]:
len(df_business_google)

7923

In [ ]:
df_business_google

### Guardamos el dataframe en parquet


In [79]:
df_business_google.to_parquet(r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\locales_gastronomicos_google.parquet')